In [1]:
import dotenv
from IPython.display import Markdown
from IPython.display import display
import os

import boto3

In [6]:
print(boto3.__version__)

1.40.21


In [2]:
dotenv.load_dotenv()
CLAUDE_MODEL_ID = os.getenv("BEDROCK_CLAUDE_MODEL_ID")
CLAUDE_CLIENT = boto3.client("bedrock-runtime",
                             region_name="ap-northeast-1")
TOOL_SPEC_1_NAME = "tool_for_make_markdown"
TOOL_SPEC_1_DESCRIPTION = (
    "あなたはMarkdownを記述する専門家で、Marpでプレゼンテーションのスライドを作成する業務に取り組んでいます。"
)  # タプルっぽいけれど1要素なので文字列扱いになる
TOOL_SPEC_1_INPUTSCHEMA_NAME_1 = "markdown_for_marp"
TOOL_SPEC_1_INPUTSCHEMA_DESCRIPTION_1 = (
    "与えられたプログラムの中身の概要を説明するためのスライドを考えて、そのスライドをMarkdownで作成してください。"
)
TOOL_SPEC_2_NAME = "tool_for_complete"
TOOL_SPEC_2_DESCRIPTION = (
    "ユーザが処理の完了を伝えてきた場合に用います。"
)
TOOL_SPEC_2_INPUTSCHEMA_NAME_1 = "for_complete"
TOOL_SPEC_2_INPUTSCHEMA_DESCRIPTION_1 = (
    "完了の旨を出力してください。"
)

In [ ]:
TARGET_CODE_FILE_PATH = r"/home/hogehoge.py"
with open(TARGET_CODE_FILE_PATH, mode="r", encoding="utf-8") as f:
    string_target_code = f.read()
USER_PROMPT = (
    "これはpythonプログラムになります。このプログラムの内容を説明するための数枚のスライドをMarkdownで出力してください。\n"
    "適宜、図やフロー等をmermaid記述で盛り込んでも構いません。\n"
    "行っている内容にフォーカスを当ててください。所感や考察などは盛り込まないでください。\n\n"
    "{a}"
).format(a=string_target_code)

In [4]:
def structured_output_tool():
    return {"toolSpec": {"name": TOOL_SPEC_1_NAME,
                         "description": TOOL_SPEC_1_DESCRIPTION,
                         "inputSchema": {"json": {"type": "object",
                                                  "properties": {TOOL_SPEC_1_INPUTSCHEMA_NAME_1: {"type": "string",
                                                                                                  "description": TOOL_SPEC_1_INPUTSCHEMA_DESCRIPTION_1}
                                                                },
                                                  "required": [TOOL_SPEC_1_INPUTSCHEMA_NAME_1]
                                                 }
                                        }
                        }
           }


def complete_tool():
    return {"toolSpec": {"name": TOOL_SPEC_2_NAME,
                         "description": TOOL_SPEC_2_DESCRIPTION,
                         "inputSchema": {"json": {"type": "object",
                                                  "properties": {TOOL_SPEC_2_INPUTSCHEMA_NAME_1: {"type": "string",
                                                                                                  "description": TOOL_SPEC_2_INPUTSCHEMA_DESCRIPTION_1}
                                                                },
                                                  "required": [TOOL_SPEC_2_INPUTSCHEMA_NAME_1]
                                                 }
                                        }
                        }
           }


def process_claude_with_function_calling(claude_message_dict_list, claude_debug_flag=False):
    claude_response = CLAUDE_CLIENT.converse(modelId=CLAUDE_MODEL_ID,
                                             messages=claude_message_dict_list,
                                             inferenceConfig={"temperature": 0.2},
                                             toolConfig={"tools": [structured_output_tool(), complete_tool()]})
    if claude_debug_flag == True:
        print(claude_response)
    result_tool_use = None
    for res in claude_response["output"]["message"]["content"]:
        if "toolUse" in res.keys():
            result_tool_use = res["toolUse"]
        else:
            continue
    return result_tool_use

In [5]:
claude_input_output_message_list = []
claude_input_message_start = {"role": "user",
                              "content": [{"text": USER_PROMPT}]
                             }
claude_input_output_message_list.append(claude_input_message_start)
while True:
    claude_result = process_claude_with_function_calling(claude_message_dict_list=claude_input_output_message_list,
                                                         claude_debug_flag=False)
    if (claude_result == None) or (claude_result["name"] == TOOL_SPEC_2_NAME):
        break
    claude_output_message = {"role": "assistant",
                             "content": [{"toolUse": claude_result}]
                            }
    claude_input_output_message_list.append(claude_output_message)
    result_markdown = claude_result["input"][TOOL_SPEC_1_INPUTSCHEMA_NAME_1]
    display(Markdown(result_markdown))  # Jupyter Notebook環境固有のメソッドでmarkdownをレンダリングして表示する
    print("\n", "*****", "\n")
    USER_PROMPT_AGAIN = input("どのようにmarkdownを変えますか？処理を完了する場合はその旨を入力してください。")
    print("\n", "*****", "\n")
    claude_input_message = {"role": "user",
                            "content": [{"toolResult": {"toolUseId": claude_result["toolUseId"],
                                                        "content": [{"text": USER_PROMPT_AGAIN}]}
                                        }]
                           }
    claude_input_output_message_list.append(claude_input_message)
print("スライド作成の処理を完了します。")
print(result_markdown)

---
marp: true
theme: default
---

# Jupyter Notebookファイルクリーンアップツール

---

# プログラムの目的

- Jupyter Notebook (.ipynb) ファイルから不要なwidgets情報を削除
- ファイルの軽量化とクリーンアップを実現

---

# 処理フロー

```mermaid
flowchart TB
    A[開始] --> B[ipynbファイルを読み込み]
    B --> C[JSONとしてパース]
    C --> D{metadataとwidgetsの<br>存在確認}
    D --> |存在する場合| E[widgets情報を削除]
    D --> |存在しない場合| F[何もしない]
    E --> G[ファイルに書き戻し]
    F --> G
    G --> H[完了メッセージ表示]
    H --> I[終了]
```

---

# 主な機能

1. **ファイル読み込み**
   - 指定されたパスのipynbファイルをUTF-8でオープン
   - JSONとしてデータを読み込み

2. **widgets情報の削除**
   - metadata内のwidgets情報を検索
   - 存在する場合は削除

3. **ファイル書き出し**
   - 処理後のデータをJSONとして書き出し
   - インデント付きで整形
   - 日本語文字のエスケープを防止

---

# 使用例

```python
clean_ipynb(path="C:\\Users\\user\\hogehoge.ipynb")
```

**出力例**:
```
C:\Users\user\hogehoge.ipynb をクリーンアップしました。
```


 ***** 



どのようにmarkdownを変えますか？処理を完了する場合はその旨を入力してください。 使用例のスライドは削除してください。



 ***** 



---
marp: true
theme: default
---

# Jupyter Notebookファイルクリーンアップツール

---

# プログラムの目的

- Jupyter Notebook (.ipynb) ファイルから不要なwidgets情報を削除
- ファイルの軽量化とクリーンアップを実現

---

# 処理フロー

```mermaid
flowchart TB
    A[開始] --> B[ipynbファイルを読み込み]
    B --> C[JSONとしてパース]
    C --> D{metadataとwidgetsの<br>存在確認}
    D --> |存在する場合| E[widgets情報を削除]
    D --> |存在しない場合| F[何もしない]
    E --> G[ファイルに書き戻し]
    F --> G
    G --> H[完了メッセージ表示]
    H --> I[終了]
```

---

# 主な機能

1. **ファイル読み込み**
   - 指定されたパスのipynbファイルをUTF-8でオープン
   - JSONとしてデータを読み込み

2. **widgets情報の削除**
   - metadata内のwidgets情報を検索
   - 存在する場合は削除

3. **ファイル書き出し**
   - 処理後のデータをJSONとして書き出し
   - インデント付きで整形
   - 日本語文字のエスケープを防止


 ***** 



どのようにmarkdownを変えますか？処理を完了する場合はその旨を入力してください。 ありがとうございます。これでokです。



 ***** 

スライド作成の処理を完了します。
---
marp: true
theme: default
---

# Jupyter Notebookファイルクリーンアップツール

---

# プログラムの目的

- Jupyter Notebook (.ipynb) ファイルから不要なwidgets情報を削除
- ファイルの軽量化とクリーンアップを実現

---

# 処理フロー

```mermaid
flowchart TB
    A[開始] --> B[ipynbファイルを読み込み]
    B --> C[JSONとしてパース]
    C --> D{metadataとwidgetsの<br>存在確認}
    D --> |存在する場合| E[widgets情報を削除]
    D --> |存在しない場合| F[何もしない]
    E --> G[ファイルに書き戻し]
    F --> G
    G --> H[完了メッセージ表示]
    H --> I[終了]
```

---

# 主な機能

1. **ファイル読み込み**
   - 指定されたパスのipynbファイルをUTF-8でオープン
   - JSONとしてデータを読み込み

2. **widgets情報の削除**
   - metadata内のwidgets情報を検索
   - 存在する場合は削除

3. **ファイル書き出し**
   - 処理後のデータをJSONとして書き出し
   - インデント付きで整形
   - 日本語文字のエスケープを防止
